In [0]:
bronze_df = spark.table("workspace.ecommerce.events_delta") # Step 1: Load Bronze Table
bronze_df.printSchema()

In [0]:
from pyspark.sql.functions import col, count, sum, avg, countDistinct, when
heavy_query_df = (
    bronze_df
        .groupBy("brand")
        .agg(
            count("*").alias("total_events"),
            sum(
                when(col("event_type") == "purchase", 1)
                .otherwise(0)
            ).alias("total_purchases"),
            sum(
                when(col("event_type") == "purchase", col("price"))
                .otherwise(0)
            ).alias("total_revenue"),
            countDistinct("user_id").alias("distinct_users"),
            avg("price").alias("avg_price")))

In [0]:
import time

start_time = time.time()

result_df = heavy_query_df.orderBy(col("total_revenue").desc())

display(result_df.limit(10))

end_time = time.time()

print("Baseline Execution Time (seconds):",
      round(end_time - start_time, 2))

In [0]:
heavy_query_df.explain(True)

In [0]:
start_time_2 = time.time()
result_df_2 = heavy_query_df.orderBy(col("total_revenue").desc())
display(result_df_2.limit(10))
end_time_2 = time.time()
print("Second Execution Time (seconds):",
      round(end_time_2 - start_time_2, 2))